Boosting a decision stump

In this homework you will implement your own boosting module.

Brace yourselves! This is going to be a fun and challenging assignment.

Use SFrames to do some feature engineering.
Train a boosted ensemble of decision-trees (gradient boosted trees) on the lending club dataset.
Predict whether a loan will default along with prediction probabilities (on a validation set).
Evaluate the trained model and compare it with a baseline.
Find the most positive and negative loans using the learned model.
Explore how the number of trees influences classification performance.



In [1036]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from math import log
from math import exp
%matplotlib inline

In [1037]:
#We will be using a dataset from the LendingClub.
#1. Load the dataset into a data frame named loans.
#Extracting the target and the feature columns
#2. We will now repeat some of the feature processing steps that we saw in the previous assignment:
#First, we re-assign the target to have +1 as a safe (good) loan, and -1 as a risky (bad) loan.
#Next, we select four categorical features:
#grade of the loan
#the length of the loan term
#the home ownership status: own, mortgage, rent
#number of years of employment.
dataFile = r'lending-club-data.csv'
#1. Load in the LendingClub dataset 
loans = pd.read_csv(dataFile, header=0, low_memory=False)
#2. Reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.
#The target column (label column) of the dataset that we are interested in is 
#called bad_loans. In this column 1means a risky (bad) loan 0 means a safe loan.
#In order to make this more intuitive and consistent with the lectures, we reassign the target to be:
#+1 as a safe loan
#-1 as a risky (bad) loan
#3. We put this in a new column called safe_loans.

loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
#delete column 'bad_loans'
loans = loans.drop('bad_loans', 1)

#Exploring some features
#2. Let's quickly explore what the dataset looks like. 
#First, print out the column names to see what features we have in this dataset.
features = loans.columns.values
print('Number of features in original data file:', np.shape(features)[0])

#Selecting features
#In this assignment, we will be using a subset of features (categorical and numeric). 
#The features we will be using are described in the code comments below. 
#If you are a finance geek, the LendingClub website has a lot more details about these features.
target = 'safe_loans'
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]

#Recall from the lectures that one 
#common approach to coping with missing values is to 
#skip observations that contain missing values.

loans = loans[[target] + features].dropna()
print('Number of features in selected columns:', loans.shape[1] -1 )
#Apply one-hot encoding to loans. Your tool may have a function for one-hot encoding. 
#Alternatively, see #7 for implementation hints.

Number of features in original data file: 68
Number of features in selected columns: 4


In [1038]:
#Apply one-hot encoding to loans. 

loans = pd.get_dummies(loans)

#Load the JSON files into the lists train_idx and test_idx.
#Perform train/validation split using train_idx and test_idx. In Pandas, for instance:

print('Number of features after hot encoding:', loans.shape[1] -1 )
#print('features list after hot encoding:', loans.columns.values )

#Split data into training and validation
#8. We split the data into training data and test data.
train_idx = json.load(open(r'module-8-assignment-2-train-idx.json')) 
test_idx = json.load(open(r'module-8-assignment-2-test-idx.json'))
train_data = loans.iloc[train_idx]
test_data = loans.iloc[test_idx]

features_list = (loans.drop(target, 1)).columns.values
print(features_list)

Number of features after hot encoding: 25
['grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F' 'grade_G'
 'term_ 36 months' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']


Weighted decision trees

7. Let's modify our decision tree code from Module 5 to support weighting of individual data points.

Weighted error definition

8. Consider a model with N data points with:

Predictions ŷ_1, ..., ŷ_n
Target y_1, ..., y_n
Data point weights α_1, ..., α_n
Then the weighted error is defined by:


where 1[ y_i ≠ ŷ_i ] is an indicator function that is set to 1 if y_i ≠ ŷ_i.

Write a function to compute weight of mistakes

9. Write a function that calculates the weight of mistakes for making the "weighted-majority" predictions for a dataset. The function accepts two inputs:

labels_in_node: y_1, ..., y_n
data_weights: Data point weights α_1, ..., α_n
We are interested in computing the (total) weight of mistakes, i.e.


This quantity is analogous to the number of mistakes, except that each mistake now carries different weight. It is related to the weighted error in the following way:


The function intermediate_node_weighted_mistakes should first compute two weights:

WM(−1): weight of mistakes when all predictions are ŷ_i = −1 i.e. WM(α,−1)
WM(+1): weight of mistakes when all predictions are ŷ_i = +1 i.e. WM(α,+1)
where −1 and +1 are vectors where all values are -1 and +1 respectively.

In [1039]:
def intermediate_node_weighted_mistakes(labels_in_node, data_weights):
    # Sum the weights of all entries with label +1
    total_weight_positive = sum(data_weights[labels_in_node == +1])
    
    # Weight of mistakes for predicting all -1's is equal to the sum above
    ### YOUR CODE HERE
    weighted_mistakes_all_negative = sum(data_weights[labels_in_node == +1])/sum(data_weights)
    
    # Sum the weights of all entries with label -1
    ### YOUR CODE HERE
    total_weight_negative = sum(data_weights[labels_in_node == -1])
    
    # Weight of mistakes for predicting all +1's is equal to the sum above
    ### YOUR CODE HERE
    weighted_mistakes_all_positive = sum(data_weights[labels_in_node == -1])/sum(data_weights)
    
    # Return the tuple (weight, class_label) representing the lower of the two weights
    #    class_label should be an integer of value +1 or -1.
    # If the two weights are identical, return (weighted_mistakes_all_positive,+1)
    ### YOUR CODE HERE
    if weighted_mistakes_all_negative > weighted_mistakes_all_positive:
        return ( weighted_mistakes_all_negative, -1 )
    else:
        return ( weighted_mistakes_all_positive, +1 )

In [1040]:
#Recall that the classification error is defined as follows:
print('Quiz Question: If we set the weights α=1 for all data points, how is the weight of mistakes WM(α,ŷ) related to the classification error?')
print('same')

Quiz Question: If we set the weights α=1 for all data points, how is the weight of mistakes WM(α,ŷ) related to the classification error?
same


In [1041]:
# Function to pick best feature to split on
# We continue modifying our decision tree code from the earlier assignment to incorporate 
# weighting of individual data points. The next step is to pick the best feature to split on.
# The best_splitting_feature function is similar to the one from the earlier assignment with 
# two minor modifications:

#The function best_splitting_feature should now accept an extra parameter data_weights to 
#take account of weights of data points.
#Instead of computing the number of mistakes in the left and right side of the split, 
#we compute the weight of mistakes for both sides, add up the two weights, and divide it by 
#the total weight of the data.
def best_splitting_feature(data, features_list, target, data_weights):
    best_feature = None
    best_error = 10

    # Keep track of the best feature 
    # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.
    # Convert to float to make sure error gets computed correctly.
    num_data_points = 1.0*data.shape[0]

    for feature in features_list:
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature]==0][target]
        # The right split will have all data points where the feature value is 1
        right_split = data[data[feature]==1][target]
        # Apply the same filtering to data_weights to create left_data_weights, right_data_weights
        ## YOUR CODE HERE
        left_data_weights = data_weights[data[feature]==0]
        right_data_weights = data_weights[data[feature]==1]
        # Calculate the number of misclassified examples in the left split.

        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        # DIFFERENT HERE
        # Calculate the weight of mistakes for left and right sides
        ## YOUR CODE HERE
        left_weighted_mistakes, left_class = intermediate_node_weighted_mistakes(left_split, left_data_weights)
        right_weighted_mistakes, right_class = intermediate_node_weighted_mistakes(right_split, right_data_weights)
        # DIFFERENT HERE
        # Compute weighted error by computing
        #  ( [weight of mistakes (left)] + [weight of mistakes (right)] ) / [total weight of all data points]
        ## YOUR CODE HERE

        error = (left_weighted_mistakes + right_weighted_mistakes)/(np.sum(left_data_weights) + np.sum(right_data_weights))
        # If this is the best error we have found so far, store the feature and the error
        if error < best_error:
            best_feature = feature
            best_error = error
    # Return the best feature we found
    return best_feature

In [1042]:
#Building the tree

#12. With the above functions implemented correctly, we are now ready to build our decision tree. 
#Let us start with a function that creates a leaf node given a set of target values. 
#The create_leaf function should be analogous to the following cell:

def create_leaf(target_values, data_weights):  
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'is_leaf': True}    
    # Computed weight of mistakes.
    # Store the predicted class (1 or -1) in leaf['prediction']
    weighted_error, best_class = intermediate_node_weighted_mistakes(target_values, data_weights)
    leaf['prediction'] = best_class ## YOUR CODE HERE
    
    return leaf

In [1043]:
#13. Now write a function that learns a weighted decision tree recursively and implements 3 stopping conditions:
#All data points in a node are from the same class.
#No more features to split on.
#Stop growing the tree when the tree depth reaches max_depth.
#Since there are many steps involved, we provide you with a Python skeleton, along with explanatory comments.

def weighted_decision_tree_create(data, features, target, data_weights, current_depth = 1, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    target_values = data[target]
    print('--------------------------------------------------------------------')
    print('Subtree, depth = %s (%s data points).' % (current_depth, len(target_values)))
    # Stopping condition 1. Error is 0.
    
    if intermediate_node_weighted_mistakes(target_values, data_weights)[0] <= 1e-15:
        print('Stopping condition 1 reached.')                
        return create_leaf(target_values, data_weights)

    # Stopping condition 2. No more features.
    if remaining_features == []:
        print('Stopping condition 2 reached.')                
        return create_leaf(target_values, data_weights)    

    # Additional stopping condition (limit tree depth)
    if current_depth > max_depth:
        print('Reached maximum depth. Stopping for now.')
        return create_leaf(target_values, data_weights)

    # If all the datapoints are the same, splitting_feature will be None. Create a leaf   
    splitting_feature = best_splitting_feature(data, features, target, data_weights)
    
    remaining_features = np.setdiff1d(remaining_features, splitting_feature)
   
    left_split = data[data[splitting_feature] == 0]
    
    right_split = data[data[splitting_feature] == 1]
    
    left_data_weights = data_weights[data[splitting_feature] == 0]
    right_data_weights = data_weights[data[splitting_feature] == 1]
    
    print('Split on feature %s. (%s, %s)' % (splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print('Creating leaf node.')
        return create_leaf(left_split[target], data_weights)
    if len(right_split) == len(data):
        print('Creating leaf node.')
        return create_leaf(right_split[target], data_weights)
    
    # Repeat (recurse) on left and right subtrees
    left_tree = weighted_decision_tree_create(
        left_split, remaining_features, target, left_data_weights, current_depth + 1, max_depth)
    right_tree = weighted_decision_tree_create(
        right_split, remaining_features, target, right_data_weights, current_depth + 1, max_depth)
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [1044]:
#14. Finally, write a recursive function to count the nodes in your tree. The function should be analogous to

def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [1045]:
#Making predictions with a weighted decision tree
#15. To make a single prediction, we must start at the root and traverse down the decision 
#tree in recursive fashion. Write a function classify that makes a single prediction. 
#It should be analogous to the following:

def classify(tree, x, annotate = False):   
    # If the node is a leaf node.

    if tree['is_leaf']:
        if annotate: 
            print('At leaf, predicting %s' % tree['prediction'])
        return tree['prediction']
    else:
        # Split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
            print('Split on %s = %s' % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [1046]:
#Evaluating the tree
#16. Create a function called evaluate_classification_error. It takes in as input:
#tree (as described above)
#data (an data frame)
#The function does not change because of adding data point weights. It is analogous to this Python function:

def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x),axis=1)
    # Once you've made the predictions, calculate the classification error
    return np.sum(prediction != data[target]) / (1.0*data.shape[0])


In [1047]:
#Example: Training a weighted decision tree

#17. To build intuition on how weighted data points affect the tree being built, consider the following:

#Suppose we only care about making good predictions for the first 10 and last 10 items in train_data, 
#we assign weights:

#1 to the last 10 items
#1 to the first 10 items
#and 0 to the rest.
#Let us fit a weighted decision tree with max_depth = 2. 
#Then compute the classification error on the subset_20, i.e. the subset of data 
#points whose weight is 1 (namely the first and last 10 data points).

# Assign weights
example_data_weights =np.zeros(train_data.shape[0])                       
example_data_weights[0:10] = 1
example_data_weights[-10:data.shape[0]] = 1

# Train a weighted decision tree model.
small_data_decision_tree_subset_20 = weighted_decision_tree_create(train_data, features_list, target, example_data_weights, current_depth = 1, max_depth=2)
#18. Now, we will compute the classification error on the subset_20, i.e. 
#the subset of data points whose weight is 1 (namely the first and last 10 data points).
class_error = evaluate_classification_error(small_data_decision_tree_subset_20, train_data)
#The model small_data_decision_tree_subset_20 performs a lot better on subset_20 than on train_data.
print('Classification Error when using 20pts to train: ', class_error)

#So, what does this mean?
#The points with higher weights are the ones that are more important during the training process of 
#the weighted decision tree.
#The points with zero weights are basically ignored during training.
print('***********************************************************************************************')
print('Quiz Question: Will you get the same model as small_data_decision_tree_subset_20 if you trained a decision tree with only the 20 data points with non-zero weights from the set of points in subset_20?')
print('No')
print('***********************************************************************************************')




--------------------------------------------------------------------
Subtree, depth = 1 (37224 data points).


/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: FutureWarning: in the fu

Split on feature grade_A. (32094, 5130)
--------------------------------------------------------------------
Subtree, depth = 2 (32094 data points).
Split on feature emp_length_1 year. (29859, 2235)
--------------------------------------------------------------------
Subtree, depth = 3 (29859 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (2235 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (5130 data points).
Split on feature emp_length_1 year. (4765, 365)
--------------------------------------------------------------------
Subtree, depth = 3 (4765 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (365 data points).
Reached maximum depth. Stopping for now.
Classification Error when usign 20pts to train:  

In [1048]:
#Implementing your own Adaboost (on decision stumps)

#19. Now that we have a weighted decision tree working, it takes only a bit of work to implement Adaboost. 
#For the sake of simplicity, let us stick with decision tree stumps by training trees with max_depth=1.
#Recall from the lecture the procedure for Adaboost:
#* Start with unweighted data with α_j = 1

#* For t=1,...,T:

#Learn f_t(x) with data weights α_j
#Compute coefficient ŵ_t:

#Re-compute weights α_j

#Normalize weights α_j

#Now write your own Adaboost function. The function accepts 4 parameters:

#data: a data frame with binary features
#features: list of feature names
#target: name of target column
#num_tree_stumps: number of tree stumps to train for the ensemble
#The function should return the list of tree stumps, along with the list of corresponding tree stump weights.

def adaboost_with_tree_stumps(data, features, target, num_tree_stumps):
    # start with unweighted data
    alpha = np.ones(data.shape[0])
    weights = []
    tree_stumps = []
    target_values = data[target]
    
    for t in range(num_tree_stumps):
        print ('=====================================================')
        print ('Adaboost Iteration %d' % t)
        print ('=====================================================')        
        # Learn a weighted decision tree stump. Use max_depth=1
        tree_stump = weighted_decision_tree_create(data, features, target, data_weights=alpha, max_depth=1)
        tree_stumps.append(tree_stump)
        
        # Make predictions
        predictions = data.apply(lambda x: classify(tree_stump, x))
        
        # Produce a Boolean array indicating whether
        # each data point was correctly classified
        is_correct = predictions == target_values
        is_wrong   = predictions != target_values
        
        # Compute weighted error
        # YOUR CODE HERE
        weighted_error = np.sum(is_wrong)/(np.sum(is_wrong)+np.sum(is_correct))      
        
        # Compute model coefficient using weighted error
        # YOUR CODE HERE
        weight = 1/2 * log((1-weighted_error)/weighted_error)
        weights.append(weight)
        
        # Adjust weights on data point
        adjustment = is_correct.apply(lambda is_correct : exp(-weight) if is_correct else exp(weight))
        
        # Scale alpha by multiplying by adjustment
        # Then normalize data points weights
        ## YOUR CODE HERE 
        alpha = alpha * (is_correct * exp(-weight) + exp(weight))
        alpha = alpha / np.sum(alpha)
    
    return [weights, tree_stumps]

#Reminders
#Stump weights (ŵ) and data point weights (α) are two different concepts.
#Stump weights (ŵ) tell you how important each stump is while making predictions with the entire boosted ensemble.
#Data point weights (α) tell you how important each data point is while training a decision stump.


In [1049]:
#Training a boosted ensemble of 10 stumps

#20. Let us train an ensemble of 10 decision tree stumps with Adaboost.
#We run the adaboost_with_tree_stumps function with the following parameters:

adaboost_10 = adaboost_with_tree_stumps(train_data, features_list, target, num_tree_stumps=10)

#Making predictions
#21. Recall from the lecture that in order to make predictions, we use the following formula:
#Do the following things in a new function predict_adaboost:
#Compute the predictions f_t(x) using the t-th decision tree
#Compute ŵ_t f_t(x) by multiplying the stump_weights with the predictions f_t(x) from the decision trees
#Sum the weighted predictions over each stump in the ensemble.
#In the end, your predict_adaboost should be analogous to this Python function:

def predict_adaboost(stump_weights, tree_stumps, data):
    scores = np.zeros(data.shape[0])
    
    for i, tree_stump in enumerate(tree_stumps):
        predictions = data.apply(lambda x: classify(tree_stump, x))
        (stump_weights * predictions)/(data.shape[0])
        # Accumulate predictions on scores array
        
    return scores.apply(lambda score : +1 if score > 0 else -1)

score = predict_adaboost(adaboost_10[0], adaboost_10[1], train_data)
print(scores)

Adaboost Iteration 0
--------------------------------------------------------------------
Subtree, depth = 1 (37224 data points).


/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: FutureWarning: in the fu

Split on feature grade_A. (32094, 5130)
--------------------------------------------------------------------
Subtree, depth = 2 (32094 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (5130 data points).
Reached maximum depth. Stopping for now.


/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:37: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/jmlbeaujour/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:38: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


KeyError: ('grade_A', 'occurred at index safe_loans')

In [1051]:
5600/(5600+1900)

0.7466666666666667